Iqbal Pahlevi Amin

In [ ]:
# download datasets
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [2]:
# melakukan ekstraksi pada file zip
import zipfile, os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

import shutil
base_dir = '/tmp/rockpaperscissors'

# remove rps-cv-images folder
shutil.rmtree(os.path.join(base_dir, 'rps-cv-images'))

# remove README 
os.remove(os.path.join(base_dir, 'README_rpc-cv-images.txt'))

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest',
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.15,
    validation_split=0.4
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.4
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    base_dir, 
    target_size=(150,150), 
    color_mode='rgb',
    batch_size=32, 
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    base_dir, 
    target_size=(150,150),
    color_mode='rgb',
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='validation'
)

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [5]:
import tensorflow as tf

# create model
model = tf.keras.models.Sequential([
    # 1st layer
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    #2nd layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #3rd layer
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #4th layer
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    #output
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [6]:
# compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# create callbacks
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(
    filepath=filepath, 
    monitor='val_loss',
    verbose=1, 
    save_best_only=True,
    mode='min',
)
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
)
tensorboard = TensorBoard(
    log_dir='logs',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None,
)

callback = [checkpoint, tensorboard, early_stopping]

In [8]:
# train model
model.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=5,
    callbacks=callback,
    verbose=1
)

Epoch 1/20
25/25 [==============================] - ETA: 0s - loss: 1.0986 - accuracy: 0.3725
Epoch 1: val_loss improved from inf to 0.96700, saving model to my_best_model.epoch01-loss0.97.hdf5
25/25 [==============================] - 18s 248ms/step - loss: 1.0986 - accuracy: 0.3725 - val_loss: 0.9670 - val_accuracy: 0.5063
Epoch 2/20
25/25 [==============================] - ETA: 0s - loss: 0.9771 - accuracy: 0.5403
Epoch 2: val_loss improved from 0.96700 to 0.95843, saving model to my_best_model.epoch02-loss0.96.hdf5
25/25 [==============================] - 7s 269ms/step - loss: 0.9771 - accuracy: 0.5403 - val_loss: 0.9584 - val_accuracy: 0.5500
Epoch 3/20
25/25 [==============================] - ETA: 0s - loss: 0.8451 - accuracy: 0.6247
Epoch 3: val_loss improved from 0.95843 to 0.85206, saving model to my_best_model.epoch03-loss0.85.hdf5
25/25 [==============================] - 6s 240ms/step - loss: 0.8451 - accuracy: 0.6247 - val_loss: 0.8521 - val_accuracy: 0.6062
Epoch 4/20
25/25

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mimg
%matplotlib inline

# upload file
uploaded = files.upload()

for fn in uploaded.keys():
  # predicting images
  path = fn

  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  predictions = model.predict(images, batch_size=10)
  print(predictions)
  labels = {'paper': predictions[0][0], 'rock':predictions[0][1], 'scissors':predictions[0][2]}
  pred_label = max(labels, key=labels.get)

  plt.title(pred_label)
  plt.show()